In [1]:
# Step 1: Setup and Imports
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib
import PIL
from PIL import Image

2024-11-10 13:45:46.222044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731264346.626705  223236 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731264346.693423  223236 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 13:45:47.581458: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define image size and paths
img_height, img_width = 224, 224
batch_size = 32

# Step 2: Load Image from URL and Visualize
# Function to download the image from the URL
def download_image_from_url(url, save_path):
    urllib.request.urlretrieve(url, save_path)

In [3]:
dataset_url = "https://dl.dropboxusercontent.com/scl/fi/vmox93cwsx2vzrish3l63/land_ls_model_data_four.zip?rlkey=6uykz7bb0aoqbl8hgfkpt1t4f&dl=0?raw=1"

# Download and unzip the dataset
dataset_path = tf.keras.utils.get_file('land_ls_model_data_four.zip', origin=dataset_url, extract=True)
extracted_dir = os.path.join(os.path.dirname(dataset_path), 'land_ls_model_data_four')  # Adjust path as necessary



1483731620/1483731620 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [4]:
import zipfile
extracted_dir = os.path.join(os.path.dirname(dataset_path), 'land_ls_model_data_four')
if not os.path.exists(extracted_dir):
    os.makedirs(extracted_dir)

# Extract the ZIP file
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Check if the extraction was successful
if not os.path.exists(extracted_dir):
    print(f"Extraction failed or directory does not exist: {extracted_dir}")
else:
    print(f"Dataset extracted successfully to: {extracted_dir}")

IsADirectoryError: [Errno 21] Is a directory: '/home/rrm3nh/.keras/datasets/land_ls_model_data_four_extracted'

In [ ]:
import shutil
extracted_dir = '/home/mjp2da/.keras/datasets/land_ls_model_data_four'

# Create a 'train' directory if it doesn't exist
train_dir = os.path.join(extracted_dir, 'train')
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

# Move 'landslide' and 'non_landslide' directories into the 'train' folder
landslide_dir = os.path.join(extracted_dir, 'landslide')
non_landslide_dir = os.path.join(extracted_dir, 'non_landslide')

# Move the directories into the 'train' directory
shutil.move(landslide_dir, os.path.join(train_dir, 'landslide'))
shutil.move(non_landslide_dir, os.path.join(train_dir, 'non_landslide'))



In [ ]:
# URLs for sample images
landslide_img_one_url = "https://dl.dropboxusercontent.com/scl/fi/viovy9pcma2y6hq58ntns/NASA_GoogE_14445.jpg?rlkey=ltitymrzuarrxxogge32c1kbi&dl=0?raw=1"
landslide_img_two_url = "https://dl.dropboxusercontent.com/scl/fi/7x41si7aih0bjh1py2phy/NASA_GoogE_14445_NonSlide.jpg?rlkey=3ll97pdrjwr9b10rcvineeynm&dl=0?raw=1"

In [ ]:
# Download images
landslide_img_one_path = tf.keras.utils.get_file('NASA_GoogE_14445', origin=landslide_img_one_url)
landslide_img_two_path = tf.keras.utils.get_file('NASA_GoogE_14445_NonSlide', origin=landslide_img_two_url)

In [ ]:
def resize_show(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(300, 300))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Prediction helper function
def predict_landslide_image(image_path, model):
    img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = predictions[0][0]

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format("landslide" if score > 0.5 else "non-landslide", 100 * score if score > 0.5 else 100 * (1 - score))
    )

In [ ]:
# Visualize images
PIL.Image.open(landslide_img_one_path)
resize_show(landslide_img_one_path)

In [ ]:
PIL.Image.open(landslide_img_two_path)
resize_show(landslide_img_two_path)

In [ ]:
# Step 3: Model Setup

# Load DenseNet121 as base model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base layers

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

In [ ]:
# Create the final model
landslide_model_DenseNet = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
landslide_model_DenseNet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalize pixel values between 0 and 1
    shear_range=0.2,              # Randomly shear images
    zoom_range=0.2,               # Randomly zoom images
    horizontal_flip=True,
    validation_split=0.2  # Randomly flip images horizontally
)

# Setup for training data - Assume that you have a directory with images classified into 'landslides' and 'non-landslides'
train_generator = train_datagen.flow_from_directory(
    train_dir,          # Path to your training data
    target_size=(img_height, img_width),  # Resize images to the required input size for DenseNet
    batch_size=batch_size,
    class_mode='binary'            # Binary classification: landslide vs non-landslide
)

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,                      # Path to your training data
    target_size=(img_height, img_width),  # Resize images
    batch_size=batch_size,
    class_mode='binary',            # Binary classification: landslide vs non-landslide
    subset='validation'             # Specify subset for validation data
)

In [ ]:
# Step 4: Train the Model
epochs = 10
history = landslide_model_DenseNet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

In [ ]:
# Step 5: Plot Training and Validation Accuracy and Loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import random
train_dir = os.path.join(extracted_dir, 'train')
test_dir = os.path.join(extracted_dir, 'test')

# Create test directory and subdirectories if they don't exist
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
for class_name in ['landslide', 'non_landslide']:
    class_train_dir = os.path.join(train_dir, class_name)
    class_test_dir = os.path.join(test_dir, class_name)
    if not os.path.exists(class_test_dir):
        os.makedirs(class_test_dir)
    
    # Move 20% of the images to the test directory
    images = os.listdir(class_train_dir)
    num_test_images = int(0.2 * len(images))
    test_images = random.sample(images, num_test_images)
    
    for image_name in test_images:
        src_path = os.path.join(class_train_dir, image_name)
        dst_path = os.path.join(class_test_dir, image_name)
        shutil.move(src_path, dst_path)

print("Test dataset created successfully.")

In [ ]:
# Define the path to your test data directory
test_dir = os.path.join(extracted_dir, 'test')

# Initialize ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data

# Set up the test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',    # Ensure binary classification mode
    shuffle=False           # Do not shuffle for evaluation
)


In [ ]:
# Step 6: Evaluate the Model on Test Data
test_loss, test_accuracy = landslide_model_DenseNet.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}, Test loss: {test_loss:.2f}')

In [ ]:
# Step 7: Single Image Prediction Helper Functions

# Prediction helper function
def predict_landslide_image(image_path, model):
    img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = predictions[0][0]

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format("landslide" if score > 0.5 else "non-landslide", 100 * score if score > 0.5 else 100 * (1 - score))
    )

In [ ]:
# Resize and display helper function
def resize_show(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(300, 300))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Testing with sample images
sample_landslide_img = landslide_img_one_path
sample_non_landslide_img = landslide_img_two_path

In [ ]:
# Display and Predict for Landslide Image
print("Landslide Image Prediction:")
resize_show(sample_landslide_img)
predict_landslide_image(sample_landslide_img, landslide_model_DenseNet)

In [ ]:
# Display and Predict for Non-Landslide Image
print("Non-Landslide Image Prediction:")
resize_show(sample_non_landslide_img)
predict_landslide_image(sample_non_landslide_img, landslide_model_DenseNet)